In [1]:
import BioSimSpace as bss
import glob
import math

/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



<>:8: DeprecationWarning: invalid escape sequence \s
<>:8: DeprecationWarning: invalid escape sequence \s
/tmp/ipykernel_14658/1346869428.py:8: DeprecationWarning: invalid escape sequence \s
  get_mdp_options = lambda index: [line.split("=")[index].strip("\s\n") for line in mdp_lines]


In [33]:
def write_mdp(work_dir: str, process_name: str, options: dict):
    """
    Write an mdp files
    """
    with open(f"{work_dir}/{process_name}.mdp", "r") as mdp:
        mdp_lines = mdp.readlines()

    get_mdp_options = lambda index: [line.split("=")[index].lstrip().rstrip().strip("\n") for line in mdp_lines]
    mdp_keys = get_mdp_options(0)
    mdp_values = get_mdp_options(-1)

    change_indices = [mdp_keys.index(key) for key in options.keys() if key in mdp_keys]

    for i in change_indices:
        for key, value in options.items():
            if key in mdp_keys:
                mdp_values[i] = value
    # print(mdp_values)
    for key, value in options.items():
        if key not in mdp_keys:
            formatted_key = key
            formatted_value = str(value)
            mdp_keys.append(formatted_key)
            mdp_values.append(formatted_value) 
    
    with open(f"{work_dir}/{process_name}.mdp", "w") as mdp:
        for i in range(len(mdp_keys)):
            mdp.write(f"{mdp_keys[i]} = {mdp_values[i]}\n")


In [44]:
with open(f"../kpc2/equilibration/unbound/ligand_10/r_nvt/r_nvt.mdp", "r") as mdp:
    mdp_lines = mdp.readlines()

In [30]:
# for line in mdp_lines:
    # print(line.split())
test = mdp_lines[-2].split("=")
print(f"{test[0].rstrip()} ={test[1].rstrip()}")
#  exclude_index = [i for i in range(len(test)) if test[i] == "="][0] + 1
# test[exclude_index:]

annealing-temp = 0.01 300.00


In [34]:
write_mdp("../kpc2/equilibration/unbound/ligand_10/r_nvt/", "r_nvt", options={"new_thing": "poo", "nsteps": 20000})

In [36]:
write_mdp("../kpc2/equilibration/unbound/ligand_10/min/", "min", options={"new_thing": "poo", "nsteps": 20000})

In [3]:
ligand_path = "../kpc2/inputs/ligands/"
solvated_ligand_files = glob.glob(ligand_path + f"ligand_10_solvated.*")
solvated = bss.IO.readMolecules(solvated_ligand_files)

In [8]:
protocol = bss.Protocol.Minimisation(steps=2000)

In [35]:
process = bss.Process.Gromacs(solvated, protocol, name="min", work_dir="../kpc2/equilibration/unbound/ligand_10/min/")


In [7]:
minimised = bss.IO.readMolecules([f"../kpc2/equilibration/unbound/ligand_10/min/min_2.gro", f"../kpc2/equilibration/unbound/ligand_10/min/min.top"])


In [15]:
r_nvt_protocol = bss.Protocol.Equilibration(runtime=5*bss.Units.Time.picosecond, temperature_start=0*bss.Units.Temperature.kelvin, temperature_end=300*bss.Units.Temperature.kelvin, restraint="all")

In [14]:
r_nvt_process =bss.Process.Gromacs(minimised, r_nvt_protocol, name="r_nvt", work_dir="../kpc2/equilibration/r_nvt/")

In [3]:
with open("../kpc2/equilibration/unbound/ligand_5/min/min_1.log", "r") as file:
    min_log_lines = file.readlines()

max_force_line = [line for line in min_log_lines if "Maximum force" in line]


In [16]:
if math.floor(math.log10(float(max_force_line.split()[3]))) < 5:
    print("poo")

poo


In [10]:
max_force_line.split()[3]

'8.8811426e+03'

In [14]:
math.floor(math.log10(float(max_force_line.split()[3])))

3